# Install

In [1]:
!pip install pdfplumber

# Import

In [2]:
import pdfplumber
import pandas as pd
import logging
logging.getLogger('pdfminer.pdfinterp').setLevel(logging.ERROR)
logging.getLogger('pdfminer.layout').setLevel(logging.ERROR)
import json
import os
from typing import List, Any, Dict
import re

# fn

In [ ]:
from modules.extract_logic import clean_int
# def clean_int(val):
#     try: return int(val.strip().replace(',', '').split('.')[0])
#     except ValueError: return val

In [ ]:
from modules.process_pdf import process_pdf
# def process_pdf(input_file_path,output_file_path):
#   result = []
#   with pdfplumber.open(input_file_path) as pdf,open(output_file_path, 'w', encoding='utf-8') as outfile:
#     outfile.write(input_file_path+'\n')
#     for page_num, page in enumerate(pdf.pages, start=1):
#       table = page.extract_table()
#       if table:
#           column_len=len(table)
#           for i, row in zip(range(column_len), table):
#             row_len=len(row)
#             if i ==0:
#               prt_str=f"#--------- Page:{page_num} Rows:{row_len} Columns:{column_len} --------#"
#             #   print(prt_str)
#               outfile.write(prt_str+'\n')
#             cleaned_row = [element for element in row if element is not None and element!='']
#             if cleaned_row:
#               result.append(cleaned_row)
#               line=str(i)+" "+str(cleaned_row)
#             #   print(line)
#               outfile.write(line + '\n')
#       else:
#           prt_str=f"#--------- Page:{page_num} No table found on this page. --------#"
#           # print(prt_str)
#           outfile.write(prt_str+'\n')
#   return result

# Code

In [ ]:
input_file_path = 'OUTPUT/ITR 1-2 AY 2023-24.pdf'
output_file_path = input_file_path.replace(".pdf", "_extracted.txt")
EXTRACTED_DATA=process_pdf(input_file_path,output_file_path)

In [ ]:
CONFIG_FILE_PATH = "config/ITR1_config.json"
with open(CONFIG_FILE_PATH, 'r') as f:
    config_data = json.load(f)  

In [ ]:
result = {}
for config_key in config_data.keys():
    data = config_data[config_key]
    result[data["id"]] = {}
    KEYS = data.get("KEYS", [])
    if not KEYS:
        continue
    PATTERN = data.get("PATTERN", "").strip().strip("r'").strip("'")
    HEADER_PATTERN = data.get("HEADER_PATTERN", "").strip().strip("r'").strip("'")
    is_in_target_schedule = False
    for row in EXTRACTED_DATA:
        row_content_str = ' '.join(map(str, row))
        if re.search(HEADER_PATTERN, row_content_str, re.IGNORECASE):
            is_in_target_schedule = True
        if is_in_target_schedule:
            match = re.search(PATTERN, row_content_str, re.IGNORECASE)
            if match:
                field_type = data.get("TYPE", "STRING")
                if field_type == "NUMERIC" and data.get("EXPECTED_ROW_LEN", 0) == len(row):
                    num_keys = len(KEYS)
                    output = {key: clean_int(row[-(num_keys - i)]) for i, key in enumerate(KEYS)}
                elif field_type == "NUMERIC" and data.get("EXPECTED_ROW_LEN", 0) != len(row):
                    output = {key: clean_int(match.group(i + 1).strip()) for i, key in enumerate(KEYS)}    
                else:
                    output = {key: match.group(i + 1).strip() for i, key in enumerate(KEYS)}
                result[data["id"]] = output
                break

In [ ]:
result

In [ ]:
assessment_year = result['Assessment_Year']['Assessment_Year']
form_type = result['Form_Type']['Form_Type']
pan = result['PAN']['PAN']

In [ ]:
data = {}
for key, value in result.items():
    data[key]=None
    if isinstance(value, dict) and 'AMOUNT_CALCULATED' in value:
        data[key] = int(str(value['AMOUNT_CALCULATED']).replace(',', ''))
    else:
        data[key] = value.get(key,None)
df = pd.DataFrame.from_dict(data, orient='index', columns=[assessment_year])
df.index.name = form_type
df